In [1]:
import pandas

In [2]:
#model parameters

growth_factor = 1.18
EBITDA_margin = 0.45
tax = 0.21
capital_exp = 0.03
working_capital_increase = 0.06
EBITDA_exit_multiple = 32.70
discount_factor = 0.94 

In [3]:
dates = pandas.date_range(start='2022',end='2032',freq='Y')
dates

DatetimeIndex(['2022-12-31', '2023-12-31', '2024-12-31', '2025-12-31',
               '2026-12-31', '2027-12-31', '2028-12-31', '2029-12-31',
               '2030-12-31', '2031-12-31'],
              dtype='datetime64[ns]', freq='A-DEC')

In [4]:
#Creat empty DataFrame using Index
model_df = pandas.DataFrame(index=dates)
terminal_value_df = pandas.DataFrame()

In [5]:
#Start filling the model
#Rev_Growth
start_rev = 200
rev_projection = [start_rev*growth_factor]

for i in range(1,len(dates)):
    rev_projection.append(rev_projection[i-1]*growth_factor) #for each date, multiply rev_projection by the previous revenue and its growth factor  
    
model_df['Revenue'] = rev_projection


In [6]:
model_df['EBITDA'] = model_df['Revenue'] * EBITDA_margin

In [7]:
model_df['Taxes']= model_df['EBITDA'] * -(tax)

In [8]:
model_df['Capital Expenditures'] = model_df['Revenue'] * -capital_exp

In [9]:
model_df['Working Capital'] = model_df['Revenue'] * -working_capital_increase

In [10]:
model_df['Cash_flow'] = model_df.drop(columns=['Revenue']).sum(axis=1)
model_df.transpose().round(2)

,2022-12-31,2023-12-31,2024-12-31,2025-12-31,2026-12-31,2027-12-31,2028-12-31,2029-12-31,2030-12-31,2031-12-31
Revenue,236.00,278.48,328.61,387.76,457.55,539.91,637.09,751.77,887.09,1046.77
EBITDA,106.20,125.32,147.87,174.49,205.90,242.96,286.69,338.30,399.19,471.05
Taxes,-22.30,-26.32,-31.05,-36.64,-43.24,-51.02,-60.21,-71.04,-83.83,-98.92
Capital Expenditures,-7.08,-8.35,-9.86,-11.63,-13.73,-16.20,-19.11,-22.55,-26.61,-31.40
Working Capital,-14.16,-16.71,-19.72,-23.27,-27.45,-32.39,-38.23,-45.11,-53.23,-62.81
Cash_flow,62.66,73.94,87.24,102.95,121.48,143.35,169.15,199.60,235.52,277.92


In [11]:
discount_factors = [discount_factor**i for i in range(1,len(dates)+1)]
discount_table = pandas.DataFrame(index=discount_factors)
discount_table['present_value_FCF'] = discount_factors*model_df['Cash_flow'].values
discount_table

,present_value_FCF
0.940000,58.898520
0.883600,65.330238
0.830584,72.464300
0.780749,80.377402
0.733904,89.154614
0.689870,98.890298
0.648478,109.689119
0.609569,121.667171
0.572995,134.953226
0.538615,149.690118


In [12]:
present_value_of_projected_fcf = discount_table['present_value_FCF'].sum()

In [13]:
present_value_of_projected_fcf

981.1150060117919

In [14]:
terminal_frame = pandas.DataFrame(index=['2032'])
terminal_frame['Revenue'] = model_df['Revenue'].iloc[-1]*growth_factor
terminal_frame['EBITDA'] = terminal_frame['Revenue']*EBITDA_margin

In [15]:
terminal_frame['Terminal_valuation']=terminal_frame['EBITDA']*EBITDA_exit_multiple
terminal_frame.round(0)

,Revenue,EBITDA,Terminal_valuation
2032,1235.0,556.0,18176.0
